In [1]:
# 基本设置

# model_dir = "/mnt/g/code/pretrain_model_dir/_modelscope/qwen/Qwen2-7B-Instruct"  # 加载需要 7分半
model_dir = "/root/home/my_model/Qwen2-7B-Instruct"  # 爷受伤了, wsl2 的 io 也太差了

torch_dtype_str = "bfloat16"
# torch_dtype_str = "float16"

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from vllm import LLM, SamplingParams

llm = LLM(model_dir, dtype=torch_dtype_str, max_model_len=1024)

INFO 06-22 16:36:35 llm_engine.py:164] Initializing an LLM engine (v0.5.0.post1) with config: model='/root/home/my_model/Qwen2-7B-Instruct', speculative_config=None, tokenizer='/root/home/my_model/Qwen2-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/root/home/my_model/Qwen2-7B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 06-22 16:36:36 utils.py:494] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 06-22 16:36:41 model_runner.py:164] Loading model weights took 14.2487 GB
INFO 06-22 16:36:42 gpu_executor.py:83] # GPU blocks: 6941, # CPU blocks: 4681
INFO 06-22 16:36:42 model_runner.py:901] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-22 16:36:42 model_runner.py:905] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-22 16:36:51 model_runner.py:977] Graph capturing finished in 9 secs.


In [23]:
sampling_params = SamplingParams(temperature=0.0, max_tokens=30)


# example = "vLLM is a high-throughput and memory-efficient inference and serving engine for LLMs.\n"
example = "你好,世界"
outputs = llm.generate([example], sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.45it/s, est. speed input: 4.35 toks/s, output: 43.53 toks/s]


In [24]:
type(outputs[0])
tokenizer.decode(outputs[0].prompt_token_ids)

'你好,世界'

In [26]:
for output in outputs:
    print(output)
    print("--")
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(output.outputs)
    print("--")

    print(generated_text)
    print(output.outputs[0].token_ids)
    print(tokenizer.decode(output.outputs[0].token_ids))

RequestOutput(request_id=1, prompt='你好,世界', prompt_token_ids=[108386, 11, 99489], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='!——记2021级新生开学典礼\n\n2021年9月1日,在秋风送爽、丹桂', token_ids=[0, 8545, 40814, 17, 15, 17, 16, 52334, 102900, 107823, 113953, 271, 17, 15, 17, 16, 7948, 24, 9754, 16, 8903, 11, 18493, 100057, 99208, 36605, 102308, 5373, 100721, 100877], cumulative_logprob=-28.88200148125179, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1719046487.7759755, last_token_time=1719046487.7759755, first_scheduled_time=1719046487.7781467, first_token_time=1719046487.8039474, time_in_queue=0.0021712779998779297, finished_time=1719046488.467085), lora_request=None)
--
[CompletionOutput(index=0, text='!——记2021级新生开学典礼\n\n2021年9月1日,在秋风送爽、丹桂', token_ids=[0, 8545, 40814, 17, 15, 17, 16, 52334, 102900, 107823, 113953, 271, 17, 15, 17, 16, 7948, 24, 9754, 16, 8903, 11, 18493, 100057, 99208, 36605, 102308, 5373, 100721, 100877]